In [24]:
import findspark
findspark.init()
import pyspark # only run after findspark.init()

In [25]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.sql('''select 'spark' as hello ''')
df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



In [26]:
#import text file with 10 minutes temperature values as csv
df_weather = spark.read.option("header", "true").option("delimiter", ";").option("inferSchema", "false").csv('wetter.txt')
df_weather.printSchema()

root
 |-- STATIONS_ID: string (nullable = true)
 |-- MESS_DATUM: string (nullable = true)
 |--   QN: string (nullable = true)
 |-- PP_10: string (nullable = true)
 |-- TT_10: string (nullable = true)
 |-- TM5_10: string (nullable = true)
 |-- RF_10: string (nullable = true)
 |-- TD_10: string (nullable = true)
 |-- eor: string (nullable = true)



In [27]:
#show data frame
df_weather.show()

+-----------+------------+-----+-------+------+------+------+------+---+
|STATIONS_ID|  MESS_DATUM|   QN|  PP_10| TT_10|TM5_10| RF_10| TD_10|eor|
+-----------+------------+-----+-------+------+------+------+------+---+
|        377|202001010000|    3|   -999|   2.4|   0.3|  87.0|   0.5|eor|
|        377|202001010010|    3|   -999|   2.4|   0.4|  86.9|   0.4|eor|
|        377|202001010020|    3|   -999|   2.3|   0.3|  87.2|   0.4|eor|
|        377|202001010030|    3|   -999|   2.2|   0.2|  87.4|   0.3|eor|
|        377|202001010040|    3|   -999|   2.3|   0.2|  87.7|   0.5|eor|
|        377|202001010050|    3|   -999|   2.2|   0.1|  88.0|   0.4|eor|
|        377|202001010100|    3|   -999|   2.2|   0.2|  87.7|   0.4|eor|
|        377|202001010110|    3|   -999|   2.1|   0.1|  88.3|   0.4|eor|
|        377|202001010120|    3|   -999|   2.1|  -0.1|  88.5|   0.4|eor|
|        377|202001010130|    3|   -999|   2.0|   0.0|  88.5|   0.3|eor|
|        377|202001010140|    3|   -999|   2.1|   0

In [36]:
#Bauernregel: Das Wetter am Siebenschläfertag sieben Wochen bleiben mag.
#filter all temperature values on Siebenschläfertag (27.06.)
df = df_weather.filter(df_weather.MESS_DATUM.contains('0627')).select(df_weather.MESS_DATUM, df_weather.TT_10)
df.show(2000)

+------------+------+
|  MESS_DATUM| TT_10|
+------------+------+
|202006270000|  18.2|
|202006270010|  17.8|
|202006270020|  17.5|
|202006270030|  17.3|
|202006270040|  17.3|
|202006270050|  17.1|
|202006270100|  16.9|
|202006270110|  16.9|
|202006270120|  16.9|
|202006270130|  16.9|
|202006270140|  17.0|
|202006270150|  16.8|
|202006270200|  16.8|
|202006270210|  17.1|
|202006270220|  17.3|
|202006270230|  17.1|
|202006270240|  16.8|
|202006270250|  16.5|
|202006270300|  16.6|
|202006270310|  16.4|
|202006270320|  17.5|
|202006270330|  16.7|
|202006270340|  16.4|
|202006270350|  16.7|
|202006270400|  17.1|
|202006270410|  17.0|
|202006270420|  17.4|
|202006270430|  18.2|
|202006270440|  19.4|
|202006270450|  19.7|
|202006270500|  19.6|
|202006270510|  20.3|
|202006270520|  19.4|
|202006270530|  19.8|
|202006270540|  19.6|
|202006270550|  20.0|
|202006270600|  20.0|
|202006270610|  19.4|
|202006270620|  19.6|
|202006270630|  19.7|
|202006270640|  20.7|
|202006270650|  21.3|
|202006270

In [60]:
#count average temparute on Siebenschläfertag
df_avg = df.groupBy().agg(F.avg(df.TT_10).alias('TT_avg'))
df_avg.show()

+-----------------+
|           TT_avg|
+-----------------+
|22.23784722222222|
+-----------------+



In [59]:
#filter all temperature values 27.06. to 15.08. (Siebenschläfertag + 7 weeks)
df_weeks = df_weather.filter((df_weather["MESS_DATUM"]>='202006270000') & (df_weather["MESS_DATUM"]<='202008152350')).select(df_weather.MESS_DATUM, df_weather.TT_10)
df_weeks.show(3000)

+------------+------+
|  MESS_DATUM| TT_10|
+------------+------+
|202006270000|  18.2|
|202006270010|  17.8|
|202006270020|  17.5|
|202006270030|  17.3|
|202006270040|  17.3|
|202006270050|  17.1|
|202006270100|  16.9|
|202006270110|  16.9|
|202006270120|  16.9|
|202006270130|  16.9|
|202006270140|  17.0|
|202006270150|  16.8|
|202006270200|  16.8|
|202006270210|  17.1|
|202006270220|  17.3|
|202006270230|  17.1|
|202006270240|  16.8|
|202006270250|  16.5|
|202006270300|  16.6|
|202006270310|  16.4|
|202006270320|  17.5|
|202006270330|  16.7|
|202006270340|  16.4|
|202006270350|  16.7|
|202006270400|  17.1|
|202006270410|  17.0|
|202006270420|  17.4|
|202006270430|  18.2|
|202006270440|  19.4|
|202006270450|  19.7|
|202006270500|  19.6|
|202006270510|  20.3|
|202006270520|  19.4|
|202006270530|  19.8|
|202006270540|  19.6|
|202006270550|  20.0|
|202006270600|  20.0|
|202006270610|  19.4|
|202006270620|  19.6|
|202006270630|  19.7|
|202006270640|  20.7|
|202006270650|  21.3|
|202006270

In [63]:
#schow average temperature 7 weeks after Siebenschläfertag
df_weeks_avg = df_weeks.groupBy().agg(F.avg(df.TT_10).alias('TT_weeks_avg'))
df_weeks_avg.show()

+------------------+
|      TT_weeks_avg|
+------------------+
|21.799333333333266|
+------------------+

